In [1]:
from plotly.offline import init_notebook_mode
from hcve_lib.feature_importance import run_permutation_importance
from hcve_lib.splitting import get_lco_splits
from hcve_lib.utils import notebook_init

notebook_init()

from mlflow import set_tracking_uri

%autoreload 2

from deps.common import get_variables_cached
from hcve_lib.tracking import search_latest_root_completed_run
from hcve_lib.feature_importance import get_permutation_importance
from deps.pipelines import get_pipelines, LCO_GB_HYPERPARAMETERS
from utils import compute_standard_metrics
from hcve_lib.cv import cross_validate
from plotly.graph_objs import Figure
from pandas import Series
from hcve_lib.utils import transpose_dict
from plotly import express as px
import plotly.graph_objs as go
from hcve_lib.functional import t
from pandas import DataFrame
from functools import partial
from hcve_lib.utils import partial2
from hcve_lib.evaluation_functions import compute_metrics_on_splits, c_index

set_tracking_uri('http://localhost:5000')

data, metadata, X, y = get_variables_cached()
init_notebook_mode(connected=True)

## ASCOT → FLEMENGHO (Coxnet)

In [ ]:
run, result = search_latest_root_completed_run('optimized_lm', 'coxnet')

fig = run_permutation_importance(
    result[('ASCOT', 'FLEMENGHO')],
    X,
    y,
    metadata,
    train_importance=True,
    n_repeats=10,
    limit=10,
)

## ASCOT → FLEMENGHO (GB)

In [6]:
run, result = search_latest_root_completed_run('optimized_lm', 'gb')
fig = run_permutation_importance(
    result[('ASCOT', 'HVC')],
    X,
    y,
    metadata,
    train_importance=True,
    n_repeats=10,
    limit=10,
)
